In [1]:
import cv2
import time
import pandas as pd
from collections import defaultdict
from ultralytics import YOLO

# Parámetros y rutas
VIDEOPATH = 'plates_test.MP4'
OUTPUTVIDEO = 'resultado.mp4'
OUTPUTCSV = 'detecciones.csv'

# Carga modelos
print("Cargando modelos...")
model_objects = YOLO("yolo11s.pt")       # Modelo para personas y vehículos
model_plates = YOLO("yolov11s_best.pt")   # Modelo matrícula

# Definiciones
VEHICLE_CLASSES = [2, 3, 5, 7]  # Clases vehículo (car, bus, etc)
PERSON_CLASS = 0
ALL_CLASSES = VEHICLE_CLASSES + [PERSON_CLASS]

class_counts = defaultdict(set)
detections_data = []
plate_cache = {}  # Almacena: {track_id: (matricula_info, last_frame_num, bbox)}

def detect_plate_in_vehicle_frame(frame, vehicle_box, frame_num):
    x1, y1, x2, y2 = map(int, vehicle_box)
    # Recorte vehículo + validación límites
    h, w = frame.shape[:2]
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w, x2), min(h, y2)
    if x2 <= x1 or y2 <= y1:
        return None
    vehicle_crop = frame[y1:y2, x1:x2]
    if vehicle_crop.size == 0:
        return None
    
    # Detección placa en recorte vehículo
    plate_results = model_plates(vehicle_crop, conf=0.4, imgsz=320, verbose=False)
    if len(plate_results[0].boxes) == 0:
        return None
    
    # Mejor caja por confianza
    best_idx = plate_results[0].boxes.conf.argmax()
    plate_box = plate_results[0].boxes[best_idx]
    px1, py1, px2, py2 = map(int, plate_box.xyxy[0])
    coords = (x1 + px1, y1 + py1, x1 + px2, y1 + py2)
    conf = float(plate_box.conf[0])
    
    # Texto OCR nominal (sólo demo, sustituir por OCR real si se tiene)
    plate_text = "MATRICULA"  # Aquí se puede integrar OCR real
    
    return {'coords': coords, 'conf': conf, 'text': plate_text, 'frame': frame_num}

def rectangles_iou(boxA, boxB):
    # Calcula el IoU entre 2 cajas: boxA y boxB = (x1,y1,x2,y2)
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea) if (boxAArea + boxBArea - interArea) > 0 else 0
    return iou

def reidentify_track(new_box, new_plate_info, frame_num, max_iou=0.5, max_frame_gap=30):
    # Intentar encontrar un track_id anterior para la nueva detección usando matrícula y posición.
    for tid, (cached_plate, last_frame, cached_box) in plate_cache.items():
        if frame_num - last_frame > max_frame_gap:
            continue  # Muy viejo, descartar
        
        # Comparar matrícula si existe
        if new_plate_info and cached_plate:
            if new_plate_info['text'] == cached_plate['text']:
                # Matricula coincide: es el mismo objeto (vehículo)
                return tid
        
        # Sin matrícula o no coincide, comparar bounding boxes (IoU)
        iou = rectangles_iou(new_box, cached_box)
        if iou > max_iou:
            return tid  # Es el mismo objeto con movimiento razonable
    
    return None  # No encontrado

# Abre vídeo y prepara salida
cap = cv2.VideoCapture(VIDEOPATH)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(OUTPUTVIDEO, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

frame_idx = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    results = model_objects.track(frame, persist=True, conf=0.4, classes=ALL_CLASSES, imgsz=640)
    
    if results[0].boxes is not None and len(results[0].boxes) > 0:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy().astype(int)
        confs = results[0].boxes.conf.cpu().numpy()
        track_ids = results[0].boxes.id.cpu().numpy().astype(int) if results[0].boxes.id is not None else range(len(boxes))
        
        for box, cls, conf, tid in zip(boxes, classes, confs, track_ids):
            x1, y1, x2, y2 = map(int, box)
            obj_type = model_objects.names[cls]
            
            plate_info = None
            
            if cls in VEHICLE_CLASSES:
                if frame_idx % 5 == 0:
                    plate_info = detect_plate_in_vehicle_frame(frame, box, frame_idx)
                
                if not plate_info and tid in plate_cache:
                    plate_info = plate_cache[tid][0]
                
                if plate_info:
                    reid_id = reidentify_track(box, plate_info, frame_idx)
                    if reid_id is not None and reid_id != tid:
                        tid = reid_id
                
                plate_cache[tid] = (plate_info, frame_idx, box)
            
            class_counts[obj_type].add(tid)
            
            detections_data.append({
                'fotograma': frame_idx,
                'tipo_objeto': obj_type,
                'confianza': round(float(conf), 3),
                'identificador_tracking': int(tid),
                'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2,
                'matricula_en_su_caso': plate_info['text'] if plate_info else '',
                'confianza_matricula': round(plate_info['conf'], 3) if plate_info else 0.0,
                'mx1': plate_info['coords'][0] if plate_info else 0,
                'my1': plate_info['coords'][1] if plate_info else 0,
                'mx2': plate_info['coords'][2] if plate_info else 0,
                'my2': plate_info['coords'][3] if plate_info else 0,
                'texto_matricula': plate_info['text'] if plate_info else ''
            })
            
            color = (0, 255, 0) if cls == PERSON_CLASS else (255, 0, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            info_text = f"ID{tid} {obj_type} {conf:.2f}"
            text_size = cv2.getTextSize(info_text, cv2.FONT_HERSHEY_SIMPLEX, 0.55, 2)[0]
            text_x, text_y = x1, max(y1 - 12, text_size[1])
            cv2.rectangle(frame, (text_x, text_y - text_size[1] - 2), (text_x + text_size[0], text_y + 2), (0, 0, 0), -1)
            cv2.putText(frame, info_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.55, color, 2)
            
            if plate_info:
                px1, py1, px2, py2 = plate_info['coords']
                cv2.rectangle(frame, (px1, py1), (px2, py2), (0, 255, 255), 2)
                plate_text = f"Plate {plate_info['conf']:.2f}"
                ptext_size = cv2.getTextSize(plate_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                ptext_x, ptext_y = px1, max(py1 - 8, ptext_size[1])
                cv2.rectangle(frame, (ptext_x, ptext_y - ptext_size[1] - 2), (ptext_x + ptext_size[0], ptext_y + 2), (0, 0, 0), -1)
                cv2.putText(frame, plate_text, (ptext_x, ptext_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    y = 30
    for obj, ids in class_counts.items():
        cv2.putText(frame, f"{obj}: {len(ids)}", (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        y += 30
    
    out.write(frame)
    frame_idx += 1

    if frame_idx % 30 == 0:
        frames_left = total_frames - frame_idx
        print(f"Frame {frame_idx}/{total_frames} - Quedan {frames_left} frames")

cap.release()
out.release()

df = pd.DataFrame(detections_data)
df.to_csv(OUTPUTCSV, index=False)

print("="*60)
print("Proceso completado")
print(f"Objetos únicos detectados:")
for obj, ids in class_counts.items():
    print(f"{obj}: {len(ids)}")

print(f"Matrículas detectadas: {len(plate_cache)}")
print("="*60)



Cargando modelos...

0: 384x640 3 cars, 1 bus, 230.0ms
Speed: 9.2ms preprocess, 230.0ms inference, 50.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 bus, 26.5ms
Speed: 3.9ms preprocess, 26.5ms inference, 7.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 bus, 25.2ms
Speed: 4.0ms preprocess, 25.2ms inference, 9.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 bus, 18.9ms
Speed: 3.0ms preprocess, 18.9ms inference, 7.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 3 cars, 1 bus, 19.6ms
Speed: 3.5ms preprocess, 19.6ms inference, 10.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 bus, 20.3ms
Speed: 3.1ms preprocess, 20.3ms inference, 5.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 bus, 22.9ms
Speed: 3.0ms preprocess, 22.9ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x6